### DSC550
##### week8 project milestone 3, Author Xin Tang 5/17/24

Requirement:
1. select, build and evaluate the model
2. train and test a model
3. join milestone 1 and 2 and 3 together

In [ ]:
# Load packages first
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import 